In [22]:
from autoprotocol.container import Well, WellGroup, Container
from autoprotocol.container_type import ContainerType
from autoprotocol.unit import Unit
import numpy as np

In [23]:
# Define the container type for 4titude 4ti-0223.
# info: http://4ti.co.uk/microplates/black-clear-bottom/96-well/
# drawing: http://4ti.co.uk/files/1614/0542/7662/4ti-0223_Marketing_Drawing.pdf
# All arguments to ContainerType are required!
capabilities = ['pipette', 'spin', 'absorbance', 'fluorescence', 'luminescence', 'incubate', 'gel_separate', 'cover', 'seal', 'stamp', 'dispense']
well_diameter = Unit(6.30, "millimeters")
well_area = np.pi * (well_diameter/2)**2
container_type = ContainerType(name='4titude 4ti-0223', is_tube=False, well_count=96, well_depth_mm=Unit(11.15, 'millimeter'), well_volume_ul=Unit(300, 'milliliter'), well_coating='polystyrene', sterile=False, capabilities=capabilities, shortname='4ti-0223', col_count=12, dead_volume_ul=Unit(20,'milliliter'), safe_min_volume_ul=Unit(50, 'milliliter'))

# Generate a unique container ID
import uuid
id = str(uuid.uuid4())

# Define the container
container = Container(name="assay-plate", id=id, container_type=container_type)

In [24]:
# Assay Parameters

Lstated = [ Unit(x, "moles/liter") for x in [20.0e-6,9.15e-6,4.18e-6,1.91e-6,0.875e-6,0.4e-6,0.183e-6,0.0837e-6,0.0383e-6,0.0175e-6,0.008e-6,0] ] # gefitinib concentration
Pstated = Unit(0.5e-6, "moles/liter")
well_volume = Unit(100, "milliliter")
receptor_name = 'Src'
ligand_name = 'gefitinib'

# Load data into well format.
filename = "../ipynbs/data-analysis/binding-assay/data/Gef_WIP_SMH_SrcBos_Extend_013015_mdfx_20150220_18.xml"
from assaytools import platereader
data = platereader.read_icontrol_xml(filename)

In [25]:
data

{'280_BottomRead': {'A1': '211',
  'A10': '101',
  'A11': '99',
  'A12': '96',
  'A2': '190',
  'A3': '175',
  'A4': '156',
  'A5': '136',
  'A6': '117',
  'A7': '108',
  'A8': '104',
  'A9': '101',
  'B1': '135',
  'B10': '95',
  'B11': '96',
  'B12': '95',
  'B2': '126',
  'B3': '108',
  'B4': '100',
  'B5': '94',
  'B6': '96',
  'B7': '93',
  'B8': '94',
  'B9': '96',
  'C1': '215',
  'C10': '94',
  'C11': '98',
  'C12': '98',
  'C2': '197',
  'C3': '177',
  'C4': '150',
  'C5': '129',
  'C6': '110',
  'C7': '105',
  'C8': '100',
  'C9': '98',
  'D1': '136',
  'D10': '93',
  'D11': '95',
  'D12': '93',
  'D2': '121',
  'D3': '108',
  'D4': '97',
  'D5': '95',
  'D6': '91',
  'D7': '96',
  'D8': '92',
  'D9': '93',
  'E1': '216',
  'E10': '97',
  'E11': '96',
  'E12': '99',
  'E2': '201',
  'E3': '179',
  'E4': '153',
  'E5': '133',
  'E6': '116',
  'E7': '103',
  'E8': '100',
  'E9': '94',
  'F1': '141',
  'F10': '89',
  'F11': '95',
  'F12': '97',
  'F2': '124',
  'F3': '108',
  'F

In [10]:
wavelength="280"
well_name="A1"
data["Abs_"+wavelength][well_name]

'0.7024'

In [26]:
# Define wells for fluorescence assay (the verbose way; we'd provide convenience methods to format the plate)
ncolumns = 12
well_group = WellGroup([])
for column in range(ncolumns):
    for row in ['A', 'B']:
        well_name = row + str(column+1)
        well = container.well(well_name)
        well.set_volume(well_volume)
        well.set_properties({'area' : well_area})

        # Set concentrations of well components
        if row == 'A':
            well.set_properties({'concentrations' : {receptor_name : Pstated, ligand_name : Lstated[column]} })
            well.set_properties({'concentration_uncertainties' : {receptor_name  : 0.10 * Pstated, ligand_name : 0.08 * Lstated[column]} })
        elif row == 'B':
            well.set_properties({'concentrations' : {ligand_name : Lstated[column]} })
            well.set_properties({'concentration_uncertainties' : {ligand_name : 0.08 * Lstated[column]} })

        # Attach plate reader data
        for wavelength in ['280', '350', '480']:
            well.set_properties({' absorbance_' + wavelength + 'nm' : data['Abs_' + wavelength][well_name] })
        emission_wavelength = '480'
        for excitation_wavelength in ['280', '350']:
            well.set_properties({'fluorescence_top_ex' + excitation_wavelength + 'nm_em' + emission_wavelength + 'nm' : data[excitation_wavelength + '_TopRead'][well_name]})
            well.set_properties({'fluorescence_bottom_ex' + excitation_wavelength + 'nm_em' + emission_wavelength + 'nm' : data[excitation_wavelength + '_BottomRead'][well_name]})

        # Add to well group
        well_group.append(well)

In [20]:
well_group

WellGroup([Well(Container(assay-plate), 0, 100.0:milliliter), Well(Container(assay-plate), 12, 100.0:milliliter), Well(Container(assay-plate), 1, 100.0:milliliter), Well(Container(assay-plate), 13, 100.0:milliliter), Well(Container(assay-plate), 2, 100.0:milliliter), Well(Container(assay-plate), 14, 100.0:milliliter), Well(Container(assay-plate), 3, 100.0:milliliter), Well(Container(assay-plate), 15, 100.0:milliliter), Well(Container(assay-plate), 4, 100.0:milliliter), Well(Container(assay-plate), 16, 100.0:milliliter), Well(Container(assay-plate), 5, 100.0:milliliter), Well(Container(assay-plate), 17, 100.0:milliliter), Well(Container(assay-plate), 6, 100.0:milliliter), Well(Container(assay-plate), 18, 100.0:milliliter), Well(Container(assay-plate), 7, 100.0:milliliter), Well(Container(assay-plate), 19, 100.0:milliliter), Well(Container(assay-plate), 8, 100.0:milliliter), Well(Container(assay-plate), 20, 100.0:milliliter), Well(Container(assay-plate), 9, 100.0:milliliter), Well(Contai

In [21]:
well_group[0].properties

{' absorbance_280nm': '0.7024',
 ' absorbance_350nm': '0.1088',
 ' absorbance_480nm': '0.0425',
 'area': Unit(31.1724531052, 'millimeter ** 2'),
 'concentration_uncertainties': {'Src': Unit(5e-08, 'mole / liter'),
  'gefitinib': Unit(1.6e-06, 'mole / liter')},
 'concentrations': {'Src': Unit(5e-07, 'mole / liter'),
  'gefitinib': Unit(2e-05, 'mole / liter')},
 'fluorescence_bottom_ex280nm_em480nm': '211',
 'fluorescence_bottom_ex350nm_em480nm': '251',
 'fluorescence_top_ex280nm_em480nm': '399',
 'fluorescence_top_ex350nm_em480nm': '156'}

In [28]:
# TODO: Analyze the well group.
# This is just a non-working stub for now.
# Create a model
from assaytools.analysis import CompetitiveBindingAnalysis
model = CompetitiveBindingAnalysis(wells=well_group, receptor_name=receptor_name, ligands=[ligand_name])
# fit the maximum a posteriori (MAP) estimate
map = model.map_fit()
# run some MCMC sampling and return the MCMC object
mcmc = model.run_mcmc()

TypeError: __init__() got an unexpected keyword argument 'ligands'